# Транзакции, ACID, MVCC, WAL.

https://edu.postgrespro.ru/dba1/dba1_05_arch_mvcc.pdf

## Перечислить требования ACID к транзакционной системе;

![acid](attest_image/trans.png)

## Каким средством реализовано выполнение этих требований в PostgreSQL?

### MVCC

![mvcc](attest_image/mvcc.png)

![snap](attest_image/snap.png)

Многоверсионность (multiversion concurrency control) – один из возможных способов организации доступа к данным. Из четырех классических требований ACID к управлению транзакциями этот механизм имеет непосредственное отношение к атомарности (транзакция либо выполняется полностью, либо полностью отменяется), согласованности (транзакция сохраняет целостность данных) и изоляции(одновременно выполняющиеся транзакции не должны влиять друг на друга).

Механизм состоит в поддержке на низком уровне одновременно нескольких версий данных. Транзакции не видят этого; они работают соснимком, который из многих версий составляет согласованную на определенный момент времени картину данных. В зависимости от уровня изоляции, снимок может определяться в момент начала транзакции (уровени repeatable read, serializable) или отдельно для каждой операции (уровень read committed).

Таким образом, транзакции смотрят на данные через призму снимков и могут видеть разную (но согласованную) информацию. Разумеется, снимок не является полной физической копией всех данных: это только логическое представление, и его можно организовать по-разному. Простой способ состоит в полном ограничении одновременного доступа: и изменений, и чтений. Но при эффективной реализации – как в Постгресе и Оракле – читающая транзакция никогда не будет заблокирована другими транзакциями, читающими или изменяющими те же данные – каждая из них будет независимо работать со своей версией. Блокироваться будут только попытки изменить данные, которые уже изменены другой транзакцией, но еще не зафиксированы.

### WAL

https://edu.postgrespro.ru/dba1/dba1_06_arch_wal.pdf

![wal](attest_image/wal.png)

что когда при сбое пропадает содержимое оперативной памяти, для восстановления необходим журнал предзаписи. Размер необходимых файлов журнала и время восстановления ограничены благодаря периодически выполняемой контрольной точке.

В предыдущих статьях мы уже посмотрели на довольно большое число важных настроек, так или иначе относящихся к журналу. В этой статье (последней в этом цикле) мы рассмотрим те вопросы настройки, которые еще не обсуждались: уровни журнала и их назначение, а также надежность и производительность журналирования.

Уровни журнала:  
    - minimal
    - replica
    - logical

# Перечислить уровни изоляции транзакций. Отличие Read Committed от Repeatable Read. В каких случаях необходимо использовать Repeatable Read?

![iso](attest_image/iso.png)

**repeatable read** -  Этот уровень удобен,
например, для отчетов, состоящих из нескольких запросов.

# Сборка мусора 

![iso](attest_image/vacum.png)

# Индексы

## Для чего нужны индексы

Ускоряют доступ к данным

## Будет ли прочитана вся таблица в случае запроса?

In [ ]:
SELECT * FROM table_name ORDER BY name LIMIT 1
# Зависит от индекса, если есть индекс то индекс уже отсортирован, и не будет вычитываться вся таблица

## Виды индексов

PostgreSQL поддерживает несколько типов индексов: 

- B-дерево, 
- хеш, 
- GiST, 
- SP-GiST, 
- GIN и 
- BRIN. 

Для разных типов индексов применяются разные алгоритмы, ориентированные на определённые типы запросов. По умолчанию команда CREATE INDEX создаёт индексы типа B-дерево, эффективные в большинстве случаев.

B-деревья могут работать в условиях на равенство и в проверках диапазонов с данными, которые можно отсортировать в некотором порядке. Точнее, планировщик запросов PostgreSQL может задействовать индекс-B-дерево, когда индексируемый столбец участвует в сравнении с одним из следующих операторов:

- "<"
- "<="
- "="
- ">="
- ">"  

При обработке конструкций, представимых как сочетание этих операторов, например BETWEEN и IN, так же может выполняться поиск по индексу-B-дереву. Кроме того, такие индексы могут использоваться и в условиях IS NULL и IS NOT NULL по индексированным столбцам.

## Почему мы не используем индекс HASH

Хеш-индексы работают только с простыми условиями равенства. Планировщик запросов может применить хеш-индекс, только если индексируемый столбец участвует в сравнении с оператором =.

Операции с хеш-индексами в настоящее время не проходят через WAL, так что после аварийной остановки базы данных может потребоваться перестроить хеш-индексы командой REINDEX. Кроме того, изменения в хеш-индексах после начальной копии не переносятся при потоковой или файловой репликации, так что в последующих запросах они будут давать неправильные ответы. По этим причинам настоятельно рекомендуется не использовать их.

## Индекс B-tree

![iso](attest_image/btree.png)

![iso](attest_image/btree2.png)

![iso](attest_image/btree3.png)

![iso](attest_image/btree4.png)

![iso](attest_image/btree5.png)

![iso](attest_image/btree6.png)

![iso](attest_image/btree7.png)

## Что делает операция VACUUM?

![iso](attest_image/vacum2.png)

![iso](attest_image/vacum3.png)

# План запроса

https://habr.com/ru/post/203386/

In [ ]:
EXPLAIN SELECT * FROM foo;

In [ ]:
QUERY PLAN
— Seq Scan on foo (cost=0.00..18334.00 rows=1000000 width=37)
(1 row)

Чтение данных из таблицы может выполняться несколькими способами. В нашем случае EXPLAIN сообщает, что используется 
- Seq Scan — последовательное, блок за блоком, чтение данных таблицы foo.
- cost Это не время, а некое сферическое в вакууме понятие, призванное оценить затратность операции. Первое значение 0.00 — затраты на получение первой строки. Второе — 18334.00 — затраты на получение всех строк.
- rows — приблизительное количество возвращаемых строк при выполнении операции Seq Scan. Это значение возвращает планировщик. В моём случае оно совпадает с реальным количеством строк в таблице.
- width — средний размер одной строки в байтах.

In [ ]:
INSERT INTO foo
  SELECT i, md5(random()::text)
  FROM generate_series(1, 10) AS i;
EXPLAIN SELECT * FROM foo;

In [ ]:
QUERY PLAN
— Seq Scan on foo (cost=0.00..18334.00 rows=1000000 width=37)
(1 row)

Значение rows не изменилось. Статистика по таблице старая. Для обновления статистики вызываем команду ANALYZE.

Что происходит при выполнении ANALYZE?

- Считывается определённое количество строк таблицы, выбранных случайным образом
- Собирается статистика значений по каждой из колонок таблицы:

Сколько строк будет считывать ANALYZE — зависит от параметра default_statistics_target.

EXPLAIN — только ожидания планировщика. Попробуем сверить их с результатами на реальных данных. Используем EXPLAIN (ANALYZE) .

In [ ]:
EXPLAIN (ANALYZE) SELECT * FROM foo;

In [ ]:
QUERY PLAN
— Seq Scan on foo (cost=0.00..18334.10 rows=1000010 width=37) (actual time=0.012..61.524 rows=1000010 loops=1)
Total runtime: 90.944 ms
(2 rows)

В выводе команды информации добавилось.
- actual time — реальное время в миллисекундах, затраченное для получения первой строки и всех строк соответственно.
- rows — реальное количество строк, полученных при Seq Scan.
- loops — сколько раз пришлось выполнить операцию Seq Scan.
- Total runtime — общее время выполнения запроса.

Пора создать индексы.

In [ ]:
CREATE INDEX ON foo(c1);
EXPLAIN SELECT * FROM foo WHERE c1 > 500;

Но ничего не поменялось!!!

In [ ]:
QUERY PLAN
— Seq Scan on foo (cost=0.00..20834.12 rows=999519 width=37)
Filter: (c1 > 500)
(2 rows)

Принудительно заставим использовать индекс, запретив Seq Scan:

In [ ]:
SET enable_seqscan TO off;
EXPLAIN (ANALYZE) SELECT * FROM foo WHERE c1 > 500;

In [ ]:
QUERY PLAN
— Index Scan using foo_c1_idx on foo (cost=0.42..34623.01 rows=999519 width=37) (actual time=0.178..1018.045 rows=999500 loops=1)
Index Cond: (c1 > 500)
Total runtime: 1434.429 ms
(3 rows)

Index Scan, Index Cond вместо Filter — используется индекс foo_c1_idx.
При выборке практически всей таблицы использование индекса только увеличивает cost и время выполнения запроса. Планировщик не глуп!

Усложним условие. Используем текстовое поле.

In [ ]:
EXPLAIN SELECT * FROM foo
        WHERE c1 < 500 AND c2 LIKE 'abcd%';

In [ ]:
QUERY PLAN
— Index Scan using foo_c1_idx on foo (cost=0.42..27.00 rows=1 width=37)
Index Cond: (c1 < 500)
Filter: (c2 ~~ 'abcd%'::text)
(3 rows)

Как видим, используется индекс foo_c1_idx для условия c1 < 500. Для c2 ~~ 'abcd%'::text используется фильтр.